In [ ]:
from tensorflow import keras
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal
import numpy as np

In [ ]:
train_input = train_input / 255

In [ ]:
from sklearn.model_selection import train_test_split
train_scaled, val_scaled, train_target, val_target = train_test_split(train_input, train_target, test_size=0.2)

In [ ]:
#from keras.api._v2.keras import activations
#from keras import layers
#model = keras.Sequential()
#model.add(keras.layers.Flatten(input_shape=(28, 28)))
#model.add(keras.layers.Dense(32, activation='relu'))
#model.add(keras.layers.Dense(64, activation='relu'))
#model.add(keras.layers.Dropout(0.3))
#model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
def build_model(n_hidden=1, n_neurons=32, out=0.3):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  for layer in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons, activation='selu'))
  model.add(keras.layers.Dropout(out))
  model.add(keras.layers.Dense(10, activation='softmax'))
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy') #398
  return model

In [ ]:
eras_rd = keras.wrappers.scikit_learn.KerasClassifier(build_model)

<ipython-input-153-51b66a618f04>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  eras_rd = keras.wrappers.scikit_learn.KerasClassifier(build_model)


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
params ={
    "n_hidden": [1, 2, 3, 4],
    "n_neurons": np.arange(1, 250),
    "out": [0.1, 0.2, 0.3],
}
mo = RandomizedSearchCV(eras_rd, params, n_iter=10, cv=3, scoring='accuracy')
mo.fit(train_scaled, train_target, epochs=100, validation_data=(val_scaled, val_target))

Epoch 1/100
1000/1000 [==============================] - 5s 4ms/step - loss: 0.6099 - val_loss: 0.4574
Epoch 2/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.4579 - val_loss: 0.4774
Epoch 3/100
1000/1000 [==============================] - 4s 4ms/step - loss: 0.4148 - val_loss: 0.4120
Epoch 4/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3903 - val_loss: 0.3759
Epoch 5/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3686 - val_loss: 0.3708
Epoch 6/100
1000/1000 [==============================] - 4s 4ms/step - loss: 0.3484 - val_loss: 0.3783
Epoch 7/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3328 - val_loss: 0.3840
Epoch 8/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3193 - val_loss: 0.3780
Epoch 9/100
 983/1000 [============================>.] - ETA: 0s - loss: 0.3112

KeyboardInterrupt: ignored

In [ ]:
model.save('modell.h5')

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint('modell.5', save_best_only=True)
early = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
history = model.fit(train_scaled, train_target, epochs=25,
validation_data=(val_scaled, val_target), callbacks=[checkpoint, early])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()